<a href="https://colab.research.google.com/github/SwaksharDeb/Activity-recognition/blob/master/HAR_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import csv
import array
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from numpy import savetxt
from sklearn.preprocessing import StandardScaler
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import tensorflow as tf
import scipy.stats as stats
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# Importing the dataset

In [71]:
dataset_user04 = pd.read_csv("final_field_user04_dataset.csv")
dataset_user07 = pd.read_csv("final_field_user07_dataset.csv")
dataset_user08 = pd.read_csv("final_field_user08_dataset.csv")
dataset_user18 = pd.read_csv("final_field_user18_dataset.csv")
dataset_user38 = pd.read_csv("final_field_user38_dataset.csv")
dataset_user51 = pd.read_csv("final_field_user51_dataset.csv")


# Data processing

In [72]:
dataset = pd.concat([dataset_user04, dataset_user07, dataset_user08, dataset_user18, dataset_user38, dataset_user51], ignore_index = True)

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1779768 entries, 0 to 1779767
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   x       float64
 1   y       float64
 2   z       float64
 3   act_id  int64  
dtypes: float64(3), int64(1)
memory usage: 54.3 MB


In [ ]:
dataset['act_id'].value_counts()

5     1074348
4      406725
7      108697
3       97370
12      47544
1       22547
6       15586
2        2698
10       2225
9        2028
Name: act_id, dtype: int64

In [98]:
dataset = pd.concat([dataset_user04, dataset_user07, dataset_user08, dataset_user18, dataset_user51], ignore_index = True)
dataset_test = dataset_user38

In [99]:
X = dataset.iloc[:, [0, 1, 2]].values
#X_test = dataset_test.iloc[:, [0, 1, 2]].values
Y = dataset.iloc[:, 3].values
#Y_test = dataset_test.iloc[:, 3].values
Y = Y - 1
#Y_test = Y_test -1

In [49]:
smote = SMOTE()
X_res,y_res=smote.fit_sample(X,Y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [50]:
X = X_res
Y = y_res

In [51]:
Counter(Y)

Counter({0: 1074348,
         1: 1074348,
         2: 1074348,
         3: 1074348,
         4: 1074348,
         5: 1074348,
         6: 1074348,
         8: 1074348,
         9: 1074348,
         11: 1074348})

## Balancing the dataset

In [ ]:
act_id_1 = dataset[dataset['act_id']==1].head(2028).copy()
act_id_2 = dataset[dataset['act_id']==2].head(2028).copy()
act_id_3 = dataset[dataset['act_id']==3].head(2028).copy()
act_id_4 = dataset[dataset['act_id']==4].head(2028).copy()
act_id_5 = dataset[dataset['act_id']==5].head(2028).copy()
act_id_6 = dataset[dataset['act_id']==6].head(2028).copy()
act_id_7 = dataset[dataset['act_id']==7].head(2028).copy()
act_id_8 = dataset[dataset['act_id']==8].head(2028).copy()
act_id_9 = dataset[dataset['act_id']==9].head(2028).copy()
act_id_10 = dataset[dataset['act_id']==10].head(2028).copy()
act_id_11 = dataset[dataset['act_id']==11].head(2028).copy()
act_id_12 = dataset[dataset['act_id']==12].head(2028).copy()

In [ ]:
balanced_data = pd.DataFrame()
balanced_data = balanced_data.append([act_id_1, act_id_2, act_id_3, act_id_4, act_id_5, act_id_6, act_id_7, act_id_8, act_id_9, act_id_10, act_id_11, act_id_12])
balanced_data.reset_index(drop=True, inplace = True)
balanced_data.shape

(20280, 4)

In [ ]:
balanced_data.head()

,x,y,z,act_id
0,-0.153,6.397,5.773,1
1,-0.153,6.397,5.773,1
2,-2.068,9.079,5.823,1
3,-4.980,7.891,0.036,1
4,-4.980,7.891,0.036,1


In [ ]:
act_id_12.head()

,x,y,z,act_id
98016,-1.494,7.508,7.098,12
98017,-1.494,7.508,7.098,12
98018,-1.494,7.508,7.098,12
98019,-1.494,7.508,7.098,12
98020,-0.996,10.496,2.565,12


In [ ]:
balanced_data['act_id'].value_counts()

12    2028
10    2028
9     2028
7     2028
6     2028
5     2028
4     2028
3     2028
2     2028
1     2028
Name: act_id, dtype: int64

## Seperating features and label

In [74]:

X = balanced_data.iloc[:, [0, 1, 2]].values
Y = balanced_data.iloc[:, 3].values

Y = Y - 1

NameError: ignored

In [ ]:
X.shape, Y.shape

((20280, 3), (20280,))

## Feature scaling

In [100]:
sc = StandardScaler()
X_scaled = sc.fit_transform(X)
#X_test_scaled = sc.fit_transform(X_test)

In [101]:
X_scaled.shape[0]

1778924

In [102]:
X_scaled

array([[-0.56721803,  0.8778758 ,  1.11696448],
       [ 0.03642697,  0.72934596,  0.99147842],
       [ 0.03642697,  0.72934596,  0.99147842],
       ...,
       [ 0.29440475,  1.48089328, -0.47191053],
       [ 0.29440475,  1.48089328, -0.47191053],
       [ 0.64664047,  0.89544385, -0.56075255]])

## Test train split

## Making the frame

In [103]:
Fs = 20
frame_size = Fs*5 # 80
hop_size = 1 #Fs*4 # 40

def get_frames(data, frame_size, hop_size, data_label):

    N_FEATURES = 3

    frames = []
    labels = []
    for i in range(0, data.shape[0] - frame_size, hop_size):
        x = data[i: i + frame_size, 0]
        y = data[i: i + frame_size, 1]
        z = data[i: i + frame_size, 2]

        # Retrieve the most often used label in this segment
        label = stats.mode(data_label[i: i + frame_size])[0][0]
        frames.append([x, y, z])
        labels.append(label)

    # Bring the segments into a better shape
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)

    return frames, labels

X_train, y_train = get_frames(X_scaled, frame_size, hop_size, Y)
#X_test, y_test = get_frames(X_test_scaled, frame_size, hop_size, Y_test)

In [56]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape 

((134293, 80, 3), (134293,), (10, 80, 3), (10,))

## Train-Test split

In [104]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 0, stratify = y_train)

In [105]:
X_train.shape, X_val.shape

((17788, 100, 3), (4448, 100, 3))

In [106]:
Counter(y_train), Counter(y_val) 

(Counter({0: 228,
          1: 28,
          2: 974,
          3: 4078,
          4: 10732,
          5: 157,
          6: 1084,
          8: 12,
          9: 20,
          11: 475}),
 Counter({0: 57,
          1: 7,
          2: 244,
          3: 1019,
          4: 2684,
          5: 39,
          6: 271,
          8: 3,
          9: 5,
          11: 119}))

In [107]:
X_train[0].shape, X_val[0].shape

((100, 3), (100, 3))

In [108]:
X_train = X_train.reshape(-1, frame_size, 3, 1)
X_val = X_val.reshape(-1, frame_size, 3, 1)
#X_test = X_test.reshape(-1, frame_size, 3, 1)

In [109]:
X_train[0].shape, X_val[0].shape

((100, 3, 1), (100, 3, 1))

In [86]:
y_train.shape, y_val.shape

((17797,), (4450,))

In [110]:
y_train[0:10]

array([4, 4, 0, 3, 3, 4, 4, 4, 4, 4])

# CNN Model

In [111]:
model = Sequential()
model.add(Conv2D(16, (2, 2), activation = 'relu', padding = 'same', input_shape = X_train[0].shape))
model.add(Dropout(0.1))

model.add(Conv2D(32, (2, 2), activation='relu', padding= 'same'))
model.add(Dropout(0.2))

#model.add(Conv2D(64, (2, 2), activation='relu', padding='same'))
#model.add(Dropout(0.2))

model.add(Flatten())

#model.add(Dense(128, activation = 'relu'))
#model.add(Dropout(0.2))

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(12, activation='softmax'))

In [112]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 100, 3, 16)        80        
_________________________________________________________________
dropout_15 (Dropout)         (None, 100, 3, 16)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 100, 3, 32)        2080      
_________________________________________________________________
dropout_16 (Dropout)         (None, 100, 3, 32)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 9600)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                614464    
_________________________________________________________________
dropout_17 (Dropout)         (None, 64)               

In [113]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [114]:
history = model.fit(X_train, y_train, epochs = 50, validation_data= (X_val, y_val), verbose=1)

Epoch 1/50
556/556 [==============================] - 13s 23ms/step - loss: 1.0660 - accuracy: 0.5969 - val_loss: 0.9050 - val_accuracy: 0.6234
Epoch 2/50
556/556 [==============================] - 13s 23ms/step - loss: 0.9531 - accuracy: 0.6124 - val_loss: 0.8774 - val_accuracy: 0.6174
Epoch 3/50
556/556 [==============================] - 13s 23ms/step - loss: 0.9225 - accuracy: 0.6167 - val_loss: 0.8699 - val_accuracy: 0.6288
Epoch 4/50
556/556 [==============================] - 13s 22ms/step - loss: 0.8985 - accuracy: 0.6258 - val_loss: 0.8567 - val_accuracy: 0.6270
Epoch 5/50
556/556 [==============================] - 12s 22ms/step - loss: 0.8763 - accuracy: 0.6286 - val_loss: 0.8387 - val_accuracy: 0.6558
Epoch 6/50
556/556 [==============================] - 12s 22ms/step - loss: 0.8639 - accuracy: 0.6344 - val_loss: 0.8395 - val_accuracy: 0.6513
Epoch 7/50
556/556 [==============================] - 12s 22ms/step - loss: 0.8540 - accuracy: 0.6373 - val_loss: 0.8232 - val_accuracy:

In [92]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy: {}".format(test_accuracy))

1/1 [==============================] - 0s 1ms/step - loss: 14.9596 - accuracy: 0.0000e+00
Test accuracy: 0.0


In [69]:
prediction = model.predict(X_test)
prediction = np.argmax(prediction, axis = 1)

In [70]:
prediction, y_test

(array([4, 4, 4, 4, 4, 3, 4, 4, 4, 4]), array([8, 8, 8, 8, 8, 8, 8, 8, 8, 8]))